In [1]:
import tensorflow as tf
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import numpy as np

In [5]:
data = load_breast_cancer()
X = data.data
y = data.target

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size =0.2,random_state = 42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [10]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(30,activation='relu',input_shape =(X_train.shape[1],)),
    tf.keras.layers.Dense(15,activation='relu'),
    tf.keras.layers.Dense(30,activation='sigmoid')
])
model.compile(optimizer = 'adam', loss = 'binary_crossentropy',metrics = ['accuracy'])
model.fit(X_train, y_train, epochs=50, batch_size = 16,validation_data = (X_test,y_test))
test_loss,test_acc = model.evaluate(X_test,y_test)
print (f'Loss pada data uji: {test_loss:.4f}')

Epoch 1/50


ValueError: Arguments `target` and `output` must have the same rank (ndim). Received: target.shape=(None,), output.shape=(None, 30)

In [27]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import re

data = pd.read_csv("reviews.csv")
print("Sample data sebelum preeprocessing")
print(data.head())

def preprocess_text(text):
    text = re.sub(r'[^a-zA-Z\s]','',text)
    text = text.lower()
    return text

data['cleaned_review']=data['reviews'].apply(preprocess_text)

print ("\nSample Data Setelah  Preprocessing:")
print(data['cleaned_review'].head())

le = LabelEncoder()
data['label'] = le.fit_transform(data['label'])
print(data['label'].head())

Sample data sebelum preeprocessing
                                             reviews  label
0  kemeja nya bagusss bgtttt😍😍😍aaaa mauuu nngisss...    1.0
1  Jahitannya sih rapi,cuman ada benang yang ikut...    0.0
2  Sesuai harga. Agak tipis tapi masih oke kok. W...    0.0
3  Wah gila sihhh sebagus itu, se worth it, se  l...    1.0
4  Kain nya bagus halus  \nTapi kok di bukak koto...    0.0

Sample Data Setelah  Preprocessing:
0    kemeja nya bagusss bgttttaaaa mauuu nngisssssk...
1    jahitannya sih rapicuman ada benang yang ikut ...
2    sesuai harga agak tipis tapi masih oke kok war...
3    wah gila sihhh sebagus itu se worth it se  lem...
4    kain nya bagus halus  \ntapi kok di bukak koto...
Name: cleaned_review, dtype: object
0    1
1    0
2    0
3    1
4    0
Name: label, dtype: int64


In [32]:
X_train, X_test, y_train, y_test = train_test_split(data['cleaned_review'],data['label'],
                                                   test_size=0.2, random_state=42)

tokenizer = Tokenizer(num_words=5000, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train)
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq= tokenizer.texts_to_sequences(X_test)

max_length = 100
X_train_padded = pad_sequences(X_train_seq,maxlen=max_length, padding='post', truncating = 'post')
X_test_padded = pad_sequences(X_test_seq,maxlen=max_length, padding='post',truncating='post')

In [36]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim = 5000, output_dim=64, input_length=max_length),
    tf.keras.layers.LSTM(64, return_sequences= True),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dense(32,activation = 'relu'),
    tf.keras.layers.Dense(3, activation = 'softmax')
])

model.compile(loss = 'sparse_categorical_crossentropy', optimizer ='adam', metrics = ['accuracy'])
model.fit(X_train_padded, y_train, epochs=5, validation_data=(X_test_padded,y_test), batch_size= 32)

Epoch 1/5
21/21 ━━━━━━━━━━━━━━━━━━━━ 5s 46ms/step - accuracy: 0.5099 - loss: 0.9685 - val_accuracy: 0.5030 - val_loss: 0.7428
Epoch 2/5
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - accuracy: 0.5394 - loss: 0.7154 - val_accuracy: 0.5030 - val_loss: 0.7198
Epoch 3/5
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - accuracy: 0.5454 - loss: 0.7070 - val_accuracy: 0.5030 - val_loss: 0.7015
Epoch 4/5
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - accuracy: 0.5476 - loss: 0.6990 - val_accuracy: 0.4970 - val_loss: 0.6985
Epoch 5/5
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - accuracy: 0.5167 - loss: 0.6957 - val_accuracy: 0.5030 - val_loss: 0.7025


In [35]:
# 7. Uji Sentimen dengan Teks Baru
new_text = ["Film ini sangat bagus, saya menyukainya"]
new_text_cleaned = [preprocess_text(text) for text in new_text]
new_text_seq = tokenizer.texts_to_sequences(new_text_cleaned)
new_text_padded = pad_sequences(new_text_seq, maxlen=max_length, padding='post', truncating='post')

# Prediksi
prediction = model.predict(new_text_padded)
predicted_label = np.argmax(prediction, axis=1)  # Mengambil kelas dengan probabilitas tertinggi

# Mengambil label dari encoded predictions
sentiment_map = {0: 'Positif', 1: 'Netral', 2: 'Negatif'}
predicted_sentiment = sentiment_map[predicted_label[0]]

print("\nHasil Uji Sentimen:")
print(f"Teks: {new_text[0]}")
print(f"Sentimen: {predicted_sentiment}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step

Hasil Uji Sentimen:
Teks: Film ini sangat bagus, saya menyukainya
Sentimen: Positif
